In [1]:
%%capture
!pip install --quiet pyvi
!pip install --quiet evaluate
!pip install -U sentence-transformers
!pip install --quiet sentencepiece tokenizers
!pip install --quiet underthesea
!pip install --quiet sentence-splitter

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer , TrainingArguments, Trainer , AutoModelForQuestionAnswering , AdamW , get_scheduler , DataCollatorWithPadding
from datasets import load_dataset
#from accelerate import Accelerator
from pyvi import ViTokenizer, ViPosTagger
import evaluate
from torch.utils.data import DataLoader
from accelerate import Accelerator
#from transformers import DataCollatorWithPadding
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit , train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from datasets import Dataset,DatasetDict
import collections
import evaluate
from huggingface_hub import notebook_login
# from underthesea import sent_tokenize ,  text_normalize , word_tokenize
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import re
import torch.nn as nn
from underthesea import word_tokenize
from underthesea import sent_tokenize ,  text_normalize , word_tokenize

SEED = 1337

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# %%capture
# checkpoint = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
# model1 = SentenceTransformer(checkpoint)
# checkpoint = "keepitreal/vietnamese-sbert"
# model2 = SentenceTransformer(checkpoint)
# checkpoint = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
# model3 = SentenceTransformer(checkpoint)
# checkpoint = "VoVanPhuc/bge-base-vi"
# model4 = SentenceTransformer(checkpoint)

In [3]:
df_train = pd.read_json("/kaggle/input/uit-train-115/ise-dsc01-train (1).json").transpose()
ds = Dataset.from_pandas(df_train)
# ds = ds.remove_columns (['verdict', 'evidence', 'domain', '__index_level_0__'])
ds

Dataset({
    features: ['context', 'claim', 'verdict', 'evidence', 'domain', '__index_level_0__'],
    num_rows: 37967
})

In [4]:
df_train.head()

,context,claim,verdict,evidence,domain
7125,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...",giao-duc
7126,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...",giao-duc
7127,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...",giao-duc
7128,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",giao-duc
7129,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...",giao-duc


In [5]:
#co nen normalize context khong ta
# def normalize_data (samples) :
#     return { "claim" :[text_normalize (sample ) for  sample in samples["claim"] ]}
# ds = ds.map (normalize_data , batched = True , remove_columns = [  "__index_level_0__"])
# ds

In [6]:
def preprocess_text(text: str) -> str:
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text
# from sentence_splitter import SentenceSplitter, split_text_into_sentences
# splitter = SentenceSplitter(language='en')

In [8]:
# %%capture
# from tqdm.auto import tqdm
# N_selected = 5
# new_context_list = []
# n = 0
# for i in tqdm ( range (len (small_train_ds)) ):
    
#     sample = small_train_ds[i]
    
#     context_arr = splitter.split(text= sample["context"])
#     context_arr = [preprocess_text (text ) for text in context_arr]
#     context_embeddings = similarity_model.encode(context_arr )
#     claim_embeddings = similarity_model.encode (sample['claim'] )
    
#     scores = [util.pytorch_cos_sim(claim_embeddings, context_embedding ) for context_embedding in context_embeddings]
    
#     pred_ind = np.argsort (scores)[-N_selected:][::-1]
#     new_context = [context_arr[ind] for ind in pred_ind]
    
#     new_context = ". ".join(new_context)
#     new_context_list.append(new_context)

## SPLIT CONTEXT SENTENCE

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("intfloat/multilingual-e5-small")

In [8]:
%%capture
N_selected = 6
def compute_context (sample) :
    # NEU NHƯ NEI thì pred sẽ là "" luôn
   
    context_arr = sample["context"].replace ("\n" ," ").split ('. ') # 0.78
    context_arr = [preprocess_text (text ) for text in context_arr]

#     context_embeddings = model1.encode(context_arr )
#     claim_embeddings = model1.encode (preprocess_text ( sample['claim'] ) )
#     scores1 = np.array ( [util.pytorch_cos_sim(claim_embeddings, context_embedding ).numpy() for context_embedding in context_embeddings] )

#     context_embeddings = model2.encode(context_arr )
#     claim_embeddings = model2.encode (preprocess_text ( sample['claim'] ) )
#     scores2 = np.array ( [util.pytorch_cos_sim(claim_embeddings, context_embedding ).numpy() for context_embedding in context_embeddings] )

#     context_embeddings = model3.encode(context_arr )
#     claim_embeddings = model3.encode (preprocess_text ( sample['claim'] ) )
#     scores3 = np.array ( [util.pytorch_cos_sim(claim_embeddings, context_embedding ).numpy() for context_embedding in context_embeddings] )

    context_embeddings = model.encode(context_arr  ,   normalize_embeddings=True)
    claim_embeddings = model.encode (preprocess_text ( sample['claim'] )  ,  normalize_embeddings=True)
    scores = np.array ( [util.pytorch_cos_sim(claim_embeddings, context_embedding ).numpy() for context_embedding in context_embeddings] )

#     scores = 0.2 * scores1 + 0.2 *  scores2 +  0.25 * scores3 + 0.35 *   scores4
    scores = scores.reshape (scores.shape[0] )
    pred_ind = np.argsort (scores)[-N_selected:][::-1]
    new_context_list = [context_arr[ind] for ind in pred_ind]
    new_context = ". ".join(new_context_list)
    return {"new_context" : new_context}

In [9]:
%%capture
ds_newtext = ds.map (compute_context)

In [ ]:
# pred_ind = np.argsort (scores)[-N_selected:][::-1]
#     new_context = [context_arr[ind] for ind in pred_ind]
    
#     new_context = ". ".join(new_context)
#     new_context_list.append(new_context)
# N_selected = 4
# def create_new_context (sample) :
#     pred_ind = np.argsort (sample["scores"])[-N_selected:][::-1]
#     new_context_list = [sample["context_arr"][ind] for ind in pred_ind]
#     new_context = ". ".join(new_context_list)
#     return {"new_context" : new_context}
# ds_new_context = ds_score.map (create_new_context)
# ds_new_context

In [ ]:
# ds_new_context[1]['new_context']

In [ ]:
# sample = ds[1]
    
# context_arr = splitter.split(text= sample["context"])
# context_embeddings = similarity_model.encode(context_arr )
# claim_embeddings = similarity_model.encode (sample['claim'] )

# scores = [util.pytorch_cos_sim(claim_embeddings, context_embedding ) for context_embedding in context_embeddings]

# pred_ind = np.argsort (scores)[-N_selected:][::-1]
# new_context = [context_arr[ind] for ind in pred_ind]

# new_context = ". ".join(new_context)
# new_context

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained ("vinai/phobert-base-v2")

In [10]:
verdict2label = {"SUPPORTED" : 2 , "REFUTED" : 0 , "NEI" : 1}
def label (samples) :
    return {"labels" : [verdict2label [sample] for sample in samples["verdict"] ]}
# def tokenize_word (samples) :
#     return {"input" : }
# def concat(samples):
#     return {"input" : [tokenizer.special_tokens_map ["cls_token"] + " "  + claim + " " + tokenizer.special_tokens_map ["sep_token"] +" " + context + " " + tokenizer.special_tokens_map ["sep_token"] for claim , context in zip ( samples["claim"] , samples["new_context"]) ]}

In [11]:
data = ds_newtext.map (label , batched = True )
# data = data.map (concat, batched = True)
# data = data.remove_columns (["context_arr" , "context_emb" , "claim_emb" ,"scores"  , "verdict" ])

  0%|          | 0/38 [00:00<?, ?ba/s]

In [12]:
data.to_json ("FULL_27K_TRAINING_RANKING_DATA_RANKING_4")

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

302379161